In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import glob
import sys
import sklearn
from sklearn.preprocessing import StandardScaler
import sklearn.preprocessing as preprocessing
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt  
np.set_printoptions(threshold=sys.maxsize)   
pd.set_option('display.max_colwidth', None)
def convert_to_csv(
        string_name, sheet_name, directory):
    
    string1 = directory
    file_name =  string1 + string_name + '.xlsx'
    read_file = pd.read_excel (file_name, sheet_name = sheet_name, header = None)
    read_file.to_csv (string1 + string_name + '.csv', index = None)
    
def change_dir(
        index):
    
    dir_win = 'J:/project files/lab_files/lump_project/data/moldir_blind_exp/'
    dir_mac = '/Volumes/Samsung_T5/project files/lab_files/lump_project/data/moldir_blind_exp/'
    if index == 1:
        directory = dir_mac
    else:
        directory = dir_win
    os.chdir(directory)
    os.listdir()
    file_name_1 = 'with_glove_all_copy'
    sheet_name1 = 'Moldir_with_glove'
    file_name_2 = 'without_glove_all_copy'
    sheet_name2 = 'Moldir_without_glove'
    convert_to_csv(file_name_1, sheet_name1, directory)
    convert_to_csv(file_name_2, sheet_name2, directory)
    
def take_out_condition(
        file, index, cut_off):
    
    return file.iloc[:, index:cut_off:3]

def correct_labels(
        data, labels):
    data_all = pd.DataFrame()
    data_s2 = pd.DataFrame()
    data_s3 = pd.DataFrame()
    # for binary Lump Presence label change   
   
    if len(labels) == 2:
        for index in range(3): 
            data_s = data.iloc[:, 3*(index):3*(index+1)]
            data_s2 = data_s.replace([1, 0], [labels[0], labels[1]])
            data_all = pd.concat([data_all, data_s2], axis = 1)
    # for size and location label change             
    elif len(labels) == 1:
        for index in range(3):
            data_s = data.iloc[:, 3*(index):3*(index+1)]
            data_s2 = data_s.replace(['1'], [labels[0]]) 
            data_all = pd.concat([data_all, data_s2], axis = 1)
    # for without lump prototypes          
    else:
        for index in range(3): 
            data_s = data.iloc[:, 3*(index):3*(index+1)]
            data_s2 = data_s.replace(["1", "0"], [labels[index], "NOLUMP"])
            data_all = pd.concat([data_all, data_s2], axis = 1)
            
    return pd.DataFrame(data_all)

def transform_to_1d(
        data):
    
    data_shape = np.shape(data)
    data_t = data.transpose()
    data_1d = (data_t.values.reshape(1, data_shape[1]*data_shape[0]))
    data_1d = pd.DataFrame(data_1d[0])
    
    return data_1d

def create_participant():
    
    participant = []
    temp2 = []
    temp = []
    for i in range(10): #  range of participants
        temp += 8*[i] # range of trials
        temp2.append(temp)
    participant += 9*temp2[0] # range of prototypes
    participant = pd.DataFrame(participant)
    
    return participant

def create_prototypes():
    
    temp = []
    for i in range(9): # range of prototypes
        temp += 80*[i]
        
    return pd.DataFrame(temp)

def correct_data_labels(
        data_all, index, LP_labels, S_labels, L_labels):
    
    shape = np.shape(data_all)
    cut_off = shape[1]//index
    if index == 1:
        start_point = shape[1]//2
    elif index == 2: 
        start_point = 0  
    data_all_LP = take_out_condition(data_all, start_point + 0, cut_off)
    data_all_S = take_out_condition(data_all, start_point + 1, cut_off)
    data_all_L = take_out_condition(data_all, start_point + 2, cut_off)

    df_LP = correct_labels(data_all_LP, LP_labels)
    df_S = correct_labels(data_all_S, S_labels)
    df_L = correct_labels(data_all_L, L_labels)
    
    df_LP_1d = transform_to_1d(df_LP)    
    df_S_1d = transform_to_1d(df_S)
    df_L_1d = transform_to_1d(df_L)
    # up to here was good
#     print(df_L_1d.values[:])

    # encode the data FOR SIZE as s -> 0 m -> 1 l -> -1 and no_lump -> -1
    # encode the data FOR LOCATION as t -> 0 m -> 1 b -> -1 and no_lump -> -1
    
#     df_LP_1d = df_LP_1d.replace(['LUMP', 'NOLUMP'], ['1', '0'])
#     df_S_1d = df_S_1d.replace(['s', 'm', 'l', 'NOLUMP'], ['0', '1', '2','-1']) 
#     df_L_1d = df_S_1d.replace(['t', 'm', 'b', 'NOLUMP'], ['0', '1', '2','-1']) 
    
#     df_LP_1d.reset_index(inplace = True, drop = True)
#     df_S_1d.reset_index(inplace = True, drop = True)
#     df_L_1d.reset_index(inplace = True, drop = True)
    
    df_participant = create_participant()
    df_prot = create_prototypes()
    df = pd.concat([df_LP_1d, df_S_1d, df_L_1d, df_participant, df_prot], axis = 1)
    df.columns = ['LP_p', 'S_p', 'L_p', 'Participant', "Prototype"]
    
    return df
############################################################
def encode(df):
    df["S_p"] = df["S_p"].replace(['s', 'm', 'l', 'NOLUMP'],
                                     ['0', '1', '2','-1']).values[:]
    df["L_p"] = df["L_p"].replace(['t', 'm', 'b', 'NOLUMP'],
                                         ['0', '1', '2','-1'])
    df["LP_p"] = df["LP_p"].replace(['LUMP', 'NOLUMP'],
                                         ['1', '0'])
    
    return pd.DataFrame(df)

def condition_data(
        index_dir, data_all):
    
    change_dir(index_dir)
    LP_lump_labels = ["LUMP", "NOLUMP"]
    S_lump_labels = ["s", "m", "l"]
    L_lump_labels = ["t", "m", "b"]
    LP_no_lump_labels = ["NOLUMP", "LUMP"]
    S_no_lump_labels = ["NOLUMP"]    
    L_no_lump_labels = ["NOLUMP"]
    
    df_l = correct_data_labels(data_all, 2, LP_lump_labels, S_lump_labels, L_lump_labels)
    df_no_l = correct_data_labels(data_all, 1,LP_no_lump_labels, S_no_lump_labels, L_no_lump_labels)
    df = pd.concat([df_l, df_no_l], axis = 0)
    
    return df
    ############################################################
def add_prot(
    df_data
    ):
    df_data_new = pd.DataFrame()
    df_all_prototype_label = df_data["LP_p"][:].map(str) + '_' + \
         + df_data["S_p"][:].map(str) + '_' + df_data["L_p"][:].map(str) 
    df_data_new = pd.concat([df_data, df_all_prototype_label], axis = 1)
    df_data_new.rename(columns={0: "Prot"}, inplace=True)
    col = df_data_new.columns
    df_data_new = df_data_new[[col[0], col[1], col[2], col[5], col[3], col[4]]]
    return df_data_new    

index_dir = 1
change_dir(index_dir)
data_all_with_glove = pd.read_csv("with_glove_all_copy.csv", sep=',')
data_all_without_glove = pd.read_csv("without_glove_all_copy.csv", sep=',')
#
df_with_glove_ = condition_data(index_dir, data_all_with_glove)
df_without_glove_ = condition_data(index_dir, data_all_without_glove)
df_with_glove_ = encode(df_with_glove_)
df_without_glove_ = encode(df_without_glove_)
df_with_glove = pd.DataFrame(add_prot(df_with_glove_))
df_without_glove = pd.DataFrame(add_prot(df_without_glove_))
# 
df_with_glove.to_csv('df_with_glove.csv', index = False)
df_without_glove.to_csv('df_without_glove.csv', index = False)

In [31]:
df_without_glove

,LP_p,S_p,L_p,Prot,Participant,Prototype
0,1,1,2,1_1_2,0,0
1,1,0,0,1_0_0,0,0
2,1,0,0,1_0_0,0,0
3,1,0,0,1_0_0,0,0
4,1,0,0,1_0_0,0,0
...,...,...,...,...,...,...
715,0,-1,-1,0_-1_-1,9,8
716,0,-1,-1,0_-1_-1,9,8
717,0,-1,-1,0_-1_-1,9,8
718,0,-1,-1,0_-1_-1,9,8


# CREATE TEST LABELS

In [30]:
# data_all_with_glove = pd.read_csv("without_glove_all_copy.csv", sep=',')
## COLLECTED RESPONSES
index_dir = 1
change_dir(index_dir)
# df_with_glove = pd.read_csv("df_with_glove.csv", sep=',')
# df_without_glove = pd.read_csv("df_without_glove.csv", sep=',')
### TEST VALUES
df_participant = create_participant()
df_prot = create_prototypes()
LP_test_l = pd.DataFrame(9*8*10*[1])
LP_test_no_l = pd.DataFrame(9*8*10*[0])
S_test_l = []
for i in range(3):
    S_test_l += 80*3*[i]
S_test_l = pd.DataFrame(S_test_l)
S_test_no_l = pd.DataFrame(9*8*10*[-1])
L_test_l = []
temp = []
for i in range(3):
    temp += 80*[i]
L_test_l += 3*temp
L_test_l = pd.DataFrame(L_test_l)
L_test_no_l = pd.DataFrame(9*8*10*[-1])
df_test_l = pd.concat([LP_test_l, S_test_l, L_test_l, df_participant, df_prot], axis = 1 )
df_test_l.columns= ['LP_p', 'S_p', 'L_p', 'Participant', "Prototype"]
df_test_no_l = pd.concat([LP_test_no_l, S_test_no_l, L_test_no_l, df_participant, df_prot], axis = 1 )
df_test_no_l.columns= ['LP_p', 'S_p', 'L_p', 'Participant', "Prototype"]
df_test = pd.concat([df_test_l, df_test_no_l], axis = 0)
df_test.reset_index(inplace = True, drop = True)
df_test = pd.DataFrame(add_prot(df_test))

df_test.to_csv('df_test.csv', index = False)

In [26]:
df_

,LP_p,S_p,L_p,Prot,Participant,Prototype
0,1,0,0,1_0_0,0,0
1,1,0,0,1_0_0,0,0
2,1,0,0,1_0_0,0,0
3,1,0,0,1_0_0,0,0
4,1,0,0,1_0_0,0,0
...,...,...,...,...,...,...
1435,0,-1,-1,0_-1_-1,9,8
1436,0,-1,-1,0_-1_-1,9,8
1437,0,-1,-1,0_-1_-1,9,8
1438,0,-1,-1,0_-1_-1,9,8
